In [1]:
import os
import pandas as pd
import re
from azure.storage.blob import BlobClient, BlobServiceClient
from dotenv import dotenv_values



#connection-string aus verstecker .env zuweisen
os.chdir('/Users/akugai/Documents/Programming-Projects/Python_Projects/CloudStorage_Pipeline')
config = dotenv_values(".env")

connection_string = config["connection_string"]



In [2]:
# Wechsel in input-folder

input_path = '/Users/akugai/Documents/Programming-Projects/Python_Projects/CloudStorage_Pipeline/data/input'

os.chdir(input_path)



In [3]:
# Liste mit allen relevanten Files wird erstellt
file_input_list = []

for filenames in os.listdir(input_path):

    file_input_list.append(filenames)


In [5]:
# Die Umwandlung in csv´s dient hauptsächlich der einfacheren (späteren) Formatierung in der SQL Server Database
output_path = '/Users/akugai/Documents/Programming-Projects/Python_Projects/CloudStorage_Pipeline/data/output'
#os.chdir(output_path)

for file in file_input_list:
    if file.endswith('.xlsx'):
        table = pd.read_excel(io=file, index_col=None)
        table.to_csv(os.path.join(output_path, re.findall('^(.*?)\.', file)[0]+'.csv') , index=False, sep=';')
    elif file.endswith('.xls'):
        table = pd.read_excel(io=file, index_col=None)
        table.to_csv(os.path.join(output_path, re.findall('^(.*?)\.', file)[0]+'.csv'), index=False, sep=';')
    elif file.endswith('.csv'):
        table = pd.read_csv(filepath_or_buffer=file, index_col=False, delimiter=';')
        table.to_csv(os.path.join(output_path, re.findall('^(.*?)\.', file)[0]+'.csv'), index=False, sep=';')
         #Alternativ könnte man im Fall einer csv auch einfach nur verschieben..
        #os.rename(os.path.join(input_path, file), os.path.join(output_path, file))
    else:
        pass

# Liste mit allen relevanten CSV-Files aus output-folder
file_output_list = []
for filenames in os.listdir(output_path):
    file_output_list.append(filenames)

In [6]:
# Das Hochladen in den MS Cloud-Storage

for filename in file_output_list:
    with open(file=os.path.join(output_path, filename), mode="rb") as data:
        try:
            # Hier werden die Files in die Cloud geladen. Sollte die File unbekannt sein, wird sie gleich hochgeladen
            blob = BlobClient.from_connection_string(conn_str=connection_string, container_name="nuernberger", blob_name=filename)
            exists = blob.exists()
            if exists == False:
                blob.upload_blob(data)
            else:
                # Dieser Schritt ist nur notwendig weil wir immer die gleichen Files hochladen!
                blob_service_client = BlobServiceClient.from_connection_string(connection_string)
                blob_client = blob_service_client.get_blob_client( container="nuernberger", blob=filename)
                blob_client.delete_blob()
                blob.upload_blob(data)
        except Exception as e:
            print(e)